# Basit Nueral Network

In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime

from tensorflow.keras.layers import Embedding, Input, Dense
import tensorflow.keras as keras

## Girdileri Okuma

In [2]:
data = {
    'tra': pd.read_csv('../input/air_visit_data.csv.gz'),
    'as': pd.read_csv('../input/air_store_info.csv.gz'),
    'hs': pd.read_csv('../input/hpg_store_info.csv.gz'),
    'ar': pd.read_csv('../input/air_reserve.csv.gz'),
    'hr': pd.read_csv('../input/hpg_reserve.csv.gz'),
    'id': pd.read_csv('../input/store_id_relation.csv.gz'),
    'tes': pd.read_csv('../input/sample_submission.csv.gz'),
    'hol': pd.read_csv('../input/date_info.csv.gz').rename(columns={'calendar_date':'visit_date'})
}

In [3]:
# Air Reserve ve HPG Reserve verilerinin birlestirilmesi
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [4]:
"""
    - Visit datetime string tipinden datetime tipine cevirme
    - Visit datetime verisinden Day Of Week bilgisini cikarma
    - Reverse datetime string tipinden datetime tipine cevirme
    - Reserve ile visit arasindaki zaman farkini ekleme
    - Exclude ???
    - 
"""
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_dow'] = data[df]['visit_datetime'].dt.dayofweek
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    # Exclude same-week reservations simple logic
    data[df] = data[df][data[df]['reserve_datetime_diff'] > data[df]['visit_dow']]
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [5]:
"""
    Training Verisinin Hazirlanmasi:
        - Visit date string tipinden datetime tipine cevirme
        - Day of Week ve Day of Year bilgilerinin visit date bilgilerinin cikartirilmasi
        - yil, ay, ve hafta bilgilerinin visit date verisinden cikartirilmasi
"""
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['doy'] = data['tra']['visit_date'].dt.dayofyear
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['week'] = data['tra']['visit_date'].dt.week
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [6]:
"""
    Test Verisinin Hazirlanmasi:
        - Visit date string tipinden datetime tipine cevirme
        - Day of Week ve Day of Year bilgilerinin visit date bilgilerinin cikartirilmasi
        - yil, ay, ve hafta bilgilerinin visit date verisinden cikartirilmasi
"""
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['doy'] = data['tes']['visit_date'].dt.dayofyear
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['week'] = data['tes']['visit_date'].dt.week
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [7]:
"""
    Tekil restorantlarin bilgilerinin cikartilmasi
"""
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

# Basit istatistiksel verilerin stores tablosuna eklenmesi
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 

In [8]:
# FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))

# Genre ve Area verilerinin basit bir label encoderdan gecirilip stores tablosuna eklenmesi

lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
# Orjinal Genre ve Area verilerinin ayni label encoderdan gecirilip stores tablosuna eklenmesi
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [9]:
# Tatil verisine day of week verisinin eklenmesi
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

# Test ve train tablolarina tatil verisinin eklenmesi
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

In [10]:
# Stores tablosunun test ve train tablolarina eklenmesi
train = pd.merge(train, stores, how='inner', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

In [11]:
# Rezerasyon bilgilerinin test ve train tablosuna eklenmesi
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

In [12]:
"""
    Rezervasyon sayilarinin ortalamalarinin tablolara eklenmesi
"""
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

In [13]:
# NEW FEATURES FROM JMBULL

# Konum bilgisinin train ve test tablolarina eklenmesi
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [14]:
# NEW FEATURES FROM Georgii Vyshnia
# Enlem ve boylam verilerinin degerlerinin toplanip yeni bir feature cikarimi
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

# Store idlere Label encoder uygulanmasi
lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
# Bos alanlarin -1 ile doldurulmasi
train = train.fillna(-1)
test = test.fillna(-1)

In [15]:
# Yarisma metriginin tanimlanmasi
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [16]:
value_col = ['holiday_flg','min_visitors','mean_visitors','median_visitors', 'max_visitors', 
             'count_observations', 
             'rs1_x','rv1_x','rs2_x','rv2_x','rs1_y','rv1_y','rs2_y','rv2_y',
             'total_reserv_sum','total_reserv_mean',
             'total_reserv_dt_diff_mean','date_int','var_max_lat','var_max_long','lon_plus_lat']

# Basit nn icerisinde kullanilacak sutunlarin belirlenmesi
nn_col = value_col + ['dow', 'year', 'month', 'air_store_id2', 'air_area_name', 'air_genre_name',
                      'air_area_name0', 'air_area_name1', 'air_area_name2', 'air_area_name3', 'air_area_name4',
                      'air_area_name5', 'air_area_name6', 'air_genre_name0', 'air_genre_name1',
                      'air_genre_name2', 'air_genre_name3', 'air_genre_name4']

X = train.copy()
X_test = test[nn_col].copy()

# Sayisal veri iceren verilerin standardize edilmesi
value_scaler = preprocessing.MinMaxScaler()
for vcol in value_col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

# Train verisinin hazirlanmasi
X_train = list(X[nn_col].T.values)
Y_train = np.log1p(X['visitors']).values
nn_train = [X_train, Y_train]
nn_test = [list(X_test[nn_col].T.values)]
print("Train and test data prepared")

Train and test data prepared


In [17]:
def get_nn_complete_model(train, hidden1_neurons=35, hidden2_neurons=15):
    # ======================= Air Store Id Embedding ======================= #
    air_store_id = Input(shape=(1,), dtype='int32', name='air_store_id')
    air_store_id_emb = Embedding(len(train['air_store_id2'].unique()) + 1, 15, input_shape=(1,),
                                 name='air_store_id_emb')(air_store_id)
    air_store_id_emb = keras.layers.Flatten(name='air_store_id_emb_flatten')(air_store_id_emb)
    
    # ===================================================================== #

    # ======================== Day of Week Embedding ======================== #
    dow = Input(shape=(1,), dtype='int32', name='dow')
    dow_emb = Embedding(8, 3, input_shape=(1,), name='dow_emb')(dow)
    dow_emb = keras.layers.Flatten(name='dow_emb_flatten')(dow_emb)
    
    # ===================================================================== #

    # ======================= Month of Week Embedding ======================= #
    month = Input(shape=(1,), dtype='int32', name='month')
    month_emb = Embedding(13, 3, input_shape=(1,), name='month_emb')(month)
    month_emb = keras.layers.Flatten(name='month_emb_flatten')(month_emb)
    
    # ===================================================================== #

    # ======================= Genre ve Area Embedding ======================= #
    air_area_name, air_genre_name = [], []
    air_area_name_emb, air_genre_name_emb = [], []
    for i in range(7):
        area_name_col = 'air_area_name' + str(i)
        air_area_name.append(Input(shape=(1,), dtype='int32', name=area_name_col))
        tmp = Embedding(len(train[area_name_col].unique()), 3, input_shape=(1,),
                        name=area_name_col + '_emb')(air_area_name[-1])
        tmp = keras.layers.Flatten(name=area_name_col + '_emb_flatten')(tmp)
        air_area_name_emb.append(tmp)

        if i > 4:
            continue
        area_genre_col = 'air_genre_name' + str(i)
        air_genre_name.append(Input(shape=(1,), dtype='int32', name=area_genre_col))
        tmp = Embedding(len(train[area_genre_col].unique()), 3, input_shape=(1,),
                        name=area_genre_col + '_emb')(air_genre_name[-1])
        tmp = keras.layers.Flatten(name=area_genre_col + '_emb_flatten')(tmp)
        air_genre_name_emb.append(tmp)

    air_genre_name_emb = keras.layers.concatenate(air_genre_name_emb)
    air_genre_name_emb = Dense(4, activation='sigmoid', name='final_air_genre_emb')(air_genre_name_emb)

    air_area_name_emb = keras.layers.concatenate(air_area_name_emb)
    air_area_name_emb = Dense(4, activation='sigmoid', name='final_air_area_emb')(air_area_name_emb)
    
    # ===================================================================== #
    
    # ======================== Area Code Embedding ======================== #
    air_area_code = Input(shape=(1,), dtype='int32', name='air_area_code')
    air_area_code_emb = Embedding(len(train['air_area_name'].unique()), 8, input_shape=(1,), name='air_area_code_emb')(air_area_code)
    air_area_code_emb = keras.layers.Flatten(name='air_area_code_emb_flatten')(air_area_code_emb)
    
    air_genre_code = Input(shape=(1,), dtype='int32', name='air_genre_code')
    air_genre_code_emb = Embedding(len(train['air_genre_name'].unique()), 5, input_shape=(1,),
                                   name='air_genre_code_emb')(air_genre_code)
    air_genre_code_emb = keras.layers.Flatten(name='air_genre_code_emb_flatten')(air_genre_code_emb)
    
    # ===================================================================== #

    # ======================= Float Fields Embedding ====================== #
    holiday_flg = Input(shape=(1,), dtype='float32', name='holiday_flg')
    year = Input(shape=(1,), dtype='float32', name='year')
    min_visitors = Input(shape=(1,), dtype='float32', name='min_visitors')
    mean_visitors = Input(shape=(1,), dtype='float32', name='mean_visitors')
    median_visitors = Input(shape=(1,), dtype='float32', name='median_visitors')
    max_visitors = Input(shape=(1,), dtype='float32', name='max_visitors')
    count_observations = Input(shape=(1,), dtype='float32', name='count_observations')
    
    # ===================================================================== #

    # =================== Reservation Statistics ========================== #
    
    rs1_x = Input(shape=(1,), dtype='float32', name='rs1_x')
    rv1_x = Input(shape=(1,), dtype='float32', name='rv1_x')
    rs2_x = Input(shape=(1,), dtype='float32', name='rs2_x')
    rv2_x = Input(shape=(1,), dtype='float32', name='rv2_x')
    rs1_y = Input(shape=(1,), dtype='float32', name='rs1_y')
    rv1_y = Input(shape=(1,), dtype='float32', name='rv1_y')
    rs2_y = Input(shape=(1,), dtype='float32', name='rs2_y')
    rv2_y = Input(shape=(1,), dtype='float32', name='rv2_y')
    total_reserv_sum = Input(shape=(1,), dtype='float32', name='total_reserv_sum')
    total_reserv_mean = Input(shape=(1,), dtype='float32', name='total_reserv_mean')
    total_reserv_dt_diff_mean = Input(shape=(1,), dtype='float32', name='total_reserv_dt_diff_mean')
    date_int = Input(shape=(1,), dtype='float32', name='date_int')
    var_max_lat = Input(shape=(1,), dtype='float32', name='var_max_lat')
    var_max_long = Input(shape=(1,), dtype='float32', name='var_max_long')
    lon_plus_lat = Input(shape=(1,), dtype='float32', name='lon_plus_lat')
    
    # ===================================================================== #
    
    # ======================= Date Embeddings ============================= #
    
    date_emb = keras.layers.concatenate([dow_emb, month_emb, year, holiday_flg])
    date_emb = Dense(5, activation='sigmoid', name='date_merged_emb')(date_emb)
    
    # ===================================================================== #
    
    # ============================ Concatenate =========================== #

    cat_layer = keras.layers.concatenate([holiday_flg, min_visitors, mean_visitors,
                    median_visitors, max_visitors, 
                    count_observations, rs1_x, rv1_x,
                    rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y,
                    total_reserv_sum, total_reserv_mean, total_reserv_dt_diff_mean,
                    date_int, var_max_lat, var_max_long, lon_plus_lat,
                    date_emb, air_area_name_emb, air_genre_name_emb,
                    air_area_code_emb, air_genre_code_emb, air_store_id_emb])
    
    # ===================================================================== #

    # ============================= Top Model ============================= #
    
    m = Dense(hidden1_neurons, name='hidden1',
             kernel_initializer=keras.initializers.RandomNormal(mean=0.0,
                            stddev=0.05, seed=None))(cat_layer)
    m = keras.layers.PReLU()(m)
    m = keras.layers.BatchNormalization()(m)
    
    m1 = Dense(hidden2_neurons, name='sub1')(m)
    m1 = keras.layers.PReLU()(m1)
    m = Dense(1, activation='relu')(m1)
    
    # ===================================================================== #
    
    # =========================== Model Build ============================= #

    inp_ten = [
        holiday_flg, min_visitors, mean_visitors, median_visitors, max_visitors, 
        count_observations,
        rs1_x, rv1_x, rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y, total_reserv_sum, total_reserv_mean,
        total_reserv_dt_diff_mean, date_int, var_max_lat, var_max_long, lon_plus_lat,
        dow, year, month, air_store_id, air_area_code, air_genre_code
    ]
    inp_ten += air_area_name
    inp_ten += air_genre_name
    model = keras.Model(inp_ten, m)
    model.compile(loss='mse', optimizer='rmsprop', metrics=['acc'])
    
    # ===================================================================== #

    return model

In [18]:
nn_model = get_nn_complete_model(train, hidden2_neurons=12)

tb = keras.callbacks.TensorBoard(log_dir='./simple_nn_logs', histogram_freq=0,
                                 write_graph=True, write_images=True, )

for i in range(5):
    nn_model.fit(nn_train[0], nn_train[1], epochs=3, verbose=1,
        batch_size=256, shuffle=True, validation_split=0.15, callbacks=[tb])
    nn_model.fit(nn_train[0], nn_train[1], epochs=8, verbose=0,
        batch_size=256, shuffle=True, callbacks=[tb])
print("Model4 trained")

nn_preds = pd.Series(nn_model.predict(nn_train[0]).reshape(-1)).clip(0, 6.8).values

print('RMSE NeuralNetwork: ', RMSLE(np.log1p(train['visitors'].values), nn_preds))

nn_preds = pd.Series(nn_model.predict(nn_test[0]).reshape(-1)).clip(0, 6.8).values

test['visitors'] = nn_preds
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
print("Model predictions done.")
# del train; del data;

/home/enis/miniconda3/envs/tf_env/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 212897 samples, validate on 37571 samples
Epoch 1/3
212897/212897 [==============================] - 15s 71us/step - loss: 0.5118 - acc: 0.0000e+00 - val_loss: 0.3942 - val_acc: 0.0000e+00
Epoch 2/3
212897/212897 [==============================] - 12s 57us/step - loss: 0.2680 - acc: 0.0000e+00 - val_loss: 0.6732 - val_acc: 0.0000e+00
Epoch 3/3
212897/212897 [==============================] - 12s 57us/step - loss: 0.2599 - acc: 0.0000e+00 - val_loss: 0.3460 - val_acc: 0.0000e+00
Train on 212897 samples, validate on 37571 samples
Epoch 1/3
212897/212897 [==============================] - 21s 96us/step - loss: 0.2501 - acc: 0.0000e+00 - val_loss: 0.2857 - val_acc: 0.0000e+00
Epoch 2/3
212897/212897 [==============================] - 20s 96us/step - loss: 0.2504 - acc: 0.0000e+00 - val_loss: 0.2560 - val_acc: 0.0000e+00
Epoch 3/3
212897/212897 [==============================] - 20s 92us/step - loss: 0.2500 - acc: 0.0000e+00 - val_loss: 0.2751 - val_acc: 0.0000e+00
Train on 212897 

In [19]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../input/*.csv')}

for k, v in dfs.items(): locals()[k] = v

date_info = data['hol'].copy()
air_visit_data = data['tra'].copy()
sample_submission = data['tes'].copy()
    
wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='visit_date', how='left')
visit_data.drop('visit_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['visit_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='visit_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub2 = sub2.fillna(-1) # for the unfound values

In [20]:
def final_visitors(x, alt=False):
    visitors_x, visitors_y = x['visitors_x'], x['visitors_y']
    if x['visitors_y'] == -1:
        return visitors_x
    else:
        return 0.7*visitors_x + 0.3*visitors_y* 1.1

sub_merge = pd.merge(sub1, sub2, on='id', how='inner')
sub_merge['visitors'] = sub_merge.apply(lambda x: final_visitors(x), axis=1)
print("Done")
sub_merge[['id', 'visitors']].to_csv('submissions/submission.csv.gz', compression='gzip', index=False)

Done
